In [44]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from sklearn.datasets import make_blobs
import random
import json
import time
from tabulate import tabulate
from sklearn.datasets import load_iris
from sklearn.mixture import GaussianMixture

from kdtree import *
from utils import *

In [45]:
savefile = "data.json"
points_count = 5000
to_plot = False
no_centres = 1

In [46]:
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt

# Initialize the parameters
from sklearn.datasets import make_circles
import matplotlib.pyplot as plt

# Initialize the parameters
noise = 0.05  # Amount of noise

# Generate the data
X, y = make_circles(n_samples=points_count, noise=noise, factor=0.5, random_state=42)

# Convert the data to a list of tuples
points = [(x, y) for x, y in X]
# points = [(round(x , 1), round(y , 1)) for x, y in X]
maxdis = math.ceil(math.log2(points_count))

In [47]:
X, Y = make_blobs(n_samples=points_count, centers=no_centres, random_state=42)
points = [(x, y) for x, y in X]

In [48]:
dcran_start_time = time.time()

In [49]:
def build():
    tree = KDTree()
    tree.root = tree.build(points)

    G = nx.Graph()

    for point in points:
        G.add_node(point , pos = point)

    neighbours = {}
    maxdis = math.ceil(math.log2(points_count))
    for point in points:
        neighbours[point] = i_neighbors(tree, point, maxdis)
    return  G,  neighbours
    

In [50]:
def merge_comps(core1 , core2, core_points_map , mst):
    pivot2 = min(core_points_map[core2], key=lambda node: euclidean_distance(node, core1))
    pivot1 = min(core_points_map[core1], key=lambda node: euclidean_distance(node, pivot2))
    mst.add_edge(pivot1, pivot2 , weight = euclidean_distance(pivot1, pivot2))
    print(f"merging {core1} and {core2} with pivot {pivot1} and {pivot2}")

In [51]:
def merge_phase(G):
    core_points_map = {}
    for component in nx.connected_components(G):
        centroid = np.mean([node for node in component], axis=0)
        closest_point = min(component, key=lambda node: euclidean_distance(node, centroid))
        core_points_map[closest_point] = component

    core_points = list(core_points_map.keys())

    minc = [float("inf")] * len(core_points[0])
    maxc = [float("-inf")] * len(core_points[0])

    for point in core_points:
        for i in range(len(point)):
            minc[i] = min(minc[i], point[i])
            maxc[i] = max(maxc[i], point[i])

    diff = [maxc[i] - minc[i] for i in range(len(minc))]
    min_diff_axis = diff.index(max(diff))

    sorted_core_points = sorted(core_points, key=lambda point: point[min_diff_axis])
    for core1 , core2 in zip(sorted_core_points, sorted_core_points[1:]):
        merge_comps(core1, core2, core_points_map, G)
        
    
    

In [52]:
def dcrun():
    G,   neighbours = build()
    print(G.number_of_nodes() , G.number_of_edges())
    k = 0
    while ( k   < maxdis) :
        print("Connected Components : " , len(list(nx.connected_components(G))))
        print(
            f"The graph has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges."
        )

        if(len(connected_components := list(nx.connected_components(G)))) == 1 : break
        for component in connected_components:
            for node in component:
                wt , pos = neighbours[node][k]
                if pos in component : continue
                G.add_edge(node , pos , weight = wt)
        k += 1
    else :
        print("merge phase")
        merge_phase(G)
        print(
            f"The graph has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges."
        )
        print("Connected Components : " , len(list(nx.connected_components(G))))
    edgecount = G.number_of_edges()
    mst = nx.minimum_spanning_tree(G)    
    print(mst.number_of_nodes() , mst.number_of_edges())
    mst_weight = sum(data["weight"] for u, v, data in mst.edges(data=True)) 
    print(
        f"Minimum Spanning Tree: {mst.number_of_nodes()} nodes, {mst.number_of_edges()} edges, Total Weight: {mst_weight}"
    )
    return mst_weight , edgecount

In [53]:
mst_weight, edgecount = dcrun()

5000 0
Connected Components :  5000
The graph has 5000 nodes and 0 edges.
Connected Components :  1541
The graph has 5000 nodes and 3459 edges.
Connected Components :  224
The graph has 5000 nodes and 5401 edges.
Connected Components :  12
The graph has 5000 nodes and 5947 edges.
Connected Components :  1
The graph has 5000 nodes and 5995 edges.
5000 4999
Minimum Spanning Tree: 5000 nodes, 4999 edges, Total Weight: 224.23808964504997


In [54]:
dcran_end_time = time.time()
dcran_elapsed_time = dcran_end_time - dcran_start_time

In [55]:
prim_start_time = time.time()

In [56]:
# Gst = nx.Graph()

# for pointi in points:
#     Gst.add_node(pointi, pos=pointi)
    
# for pointi in points:    
#     for pointj in points:
#         if pointi != pointj:
#             dis = euclidean_distance(pointi, pointj)
#             Gst.add_edge(pointi , pointj , weight=dis)

In [57]:
# Gst = nx.minimum_spanning_tree(Gst, algorithm="prim", weight="weight")
# gst_weight = sum(data["weight"] for u, v, data in Gst.edges(data=True))


In [58]:
prim_end_time = time.time()
prim_elapsed_time = prim_end_time - prim_start_time

In [59]:
import math

eprim_start_time = time.time()
def calculate_distance(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)


def prim_mst(points):
    n = len(points)
    visited = [False] * n
    min_cost = [float("inf")] * n
    parent = [None] * n

    min_cost[0] = 0
    parent[0] = -1

    for _ in range(n):
        min_index = -1
        for i in range(n):
            if not visited[i] and (
                min_index == -1 or min_cost[i] < min_cost[min_index]
            ):
                min_index = i

        visited[min_index] = True

        for i in range(n):
            if not visited[i]:
                distance = calculate_distance(points[min_index], points[i])
                if distance < min_cost[i]:
                    min_cost[i] = distance
                    parent[i] = min_index

    mst_edges = []
    for i in range(1, n):
        mst_edges.append((parent[i], i))

    return mst_edges


mst = prim_mst(points)
eprim_end_time = time.time()
eprim_elapsed_time = eprim_end_time - eprim_start_time
print("MST Weight: ", eprim_wt := sum([calculate_distance(points[u], points[v]) for u, v in mst]))
print("time taken by dcran : ", eprim_elapsed_time)

MST Weight:  221.68865919201852
time taken by dcran :  7.015679836273193


In [60]:
speedup = eprim_elapsed_time / dcran_elapsed_time
print(f"Speedup: {speedup:.2f}")

Speedup: 10.14


In [61]:
wt_error = abs(mst_weight - eprim_wt) / eprim_wt * 100
print(f"Weight Error: {wt_error}%")

Weight Error: 1.1500049043208953%


In [62]:
with open(savefile, "r") as f:
    loaded_data = json.load(f)
print(loaded_data)
currres = []
loaded_data.append(
    [
        points_count,
        no_centres,
        mst_weight,
        eprim_wt,
        edgecount,
        wt_error,
        dcran_elapsed_time,
        eprim_elapsed_time,
        speedup,
    ]
)

# Save the updated dictionary back to the JSON file
with open(savefile, "w") as f:
    json.dump(loaded_data, f)

FileNotFoundError: [Errno 2] No such file or directory: 'data.json'

In [ ]:
headers = [
    "Points",
    "Centres",
    "DCRAN Wt",
    "prims Wt",
    "Edge count",
    "Acc(%)",
    "DCRAN Time (s)",
    "STMST Time (s)",
    "Speedup",
]

# Format the data as a table using tabulate
table_str = tabulate(
    loaded_data[-15:],
    headers,
    tablefmt="pipe",
    floatfmt=(".0f", ".0f", ".1f", ".1f", ".0f", ".2f", ".2f", ".2f", ".2f"),
)
print(table_str)